In [1]:
import pandas as pd
import numpy as np
import json
import time
import matplotlib.pyplot as plt

from scipy.spatial import distance

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot , Tokenizer
from tensorflow.keras.applications.xception import Xception
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input, BatchNormalization, Dropout
from keras import Sequential, Model
from keras.layers import Lambda, Reshape, Dot
from keras.losses import CosineSimilarity
from keras.layers import merge

Get the text embeddings

In [2]:

# Opening JSON file
f = open('text_embs.json')
 
# returns JSON object as
# a dictionary
text_embeddings = json.load(f)
 
# Iterating through the
f.close()

In [3]:
text_embeddings = np.array([text_embeddings[str(i)] for i in range(1,27639)])

In [4]:
np.shape(text_embeddings)

(27638, 1200)

Get the image model to get the image embeddings

In [ ]:
#model = keras.models.load_model('cuisine_im_mod.h5')

In [ ]:
#new_model = tf.keras.Model(model.inputs, model.layers[-1].output)
#new_model.set_weights(model.get_weights())


In [5]:
temp = pd.read_csv('recipe_info.csv')

In [ ]:
#temp['id'].unique()

In [ ]:
#directory = '/Users/rpindale/Desktop/Yummly28K'
#labels = list(temp['id'].unique())


#all_images = tf.keras.utils.image_dataset_from_directory(directory = directory,
#                                            labels = labels, 
#                                            label_mode='int',
#                                            batch_size = 1, 
#                                            shuffle = False)

In [6]:
#image_embeddings = new_model.predict(all_images)

#a_file = open("image_embeddings.txt", "w")
#for row in image_embeddings:
#    np.sav I etxt(a_file, row)

#a_file.close()

#np.shape(image_embeddings) = (27638, 8192)


image_embeddings = np.loadtxt("image_embeddings.txt").reshape(27638, 8192)

In [7]:
np.shape(image_embeddings)

(27638, 8192)

In [8]:

left_input = Input(shape=(1200,))
right_input = Input(shape=(8192,))
    
premodel = Sequential()
premodel.add(Dense(1200, activation='relu', input_shape=(8192,)))
premodel.add(BatchNormalization())

model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(1200,)))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(1000, activation='relu'))
#model.add(Dropout(.15))
#model.add(BatchNormalization())
#model.add(Dense(200, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dense(50, activation='relu'))
    

encoded_l = model(left_input)
encoded_r = model(premodel(right_input))



cos_distance = keras.layers.Dot(axes=1, normalize=True)([encoded_l, encoded_r])

#cos_distance = merge([encoded_l, encoded_r], mode='cos', dot_axes=1)
cos_distance = Reshape((1,))(cos_distance)
cos_similarity = Lambda(lambda x: 1-x)(cos_distance)

#cos_similarity = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([encoded_l, encoded_r])
#siamese_net = Model([left_input, right_input], L1_Distance)
siamese_net = Model([left_input, right_input], [cos_distance])
siamese_net.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, )
siamese_net.compile(loss="mse",optimizer=optimizer)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8192)]       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1200)]       0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 1200)         9836400     input_2[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1000)         2206000     input_1[0][0]                    
                                                                 sequential[0][0]             

In [19]:
output = np.ones(27638)

siamese_net.fit(x=[text_embeddings,image_embeddings],
                y=[output], 
                batch_size=16, 
                epochs=1, 
                validation_split=.2)

KeyboardInterrupt: 

In [9]:
output = np.concatenate((np.ones(27638), np.zeros(27638)))

siamese_net.fit(x=[np.concatenate((text_embeddings, np.concatenate((text_embeddings[2:], text_embeddings[0:2])))),np.concatenate((image_embeddings, image_embeddings))],
                y=output, 
                batch_size=16, 
                epochs=1, 
                validation_split=.2)


2764/2764 [==============================] - 194s 69ms/step - loss: 0.2597 - val_loss: 0.2465


In [19]:

siamese_net.predict([text_embeddings,image_embeddings])

array([[0.65777045],
       [0.622525  ],
       [0.74331486],
       ...,
       [0.6331605 ],
       [0.72320634],
       [0.4335317 ]], dtype=float32)

In [20]:
siamese_net.predict([text_embeddings[1:7],image_embeddings[20:26]])

array([[0.58484846],
       [0.40528423],
       [0.64663357],
       [0.29711282],
       [0.15588544],
       [0.90579104]], dtype=float32)

In [18]:
preds = []
for i in range(27638): 
    preds.append(siamese_net.predict([text_embeddings[i].reshape((1,1200)),image_embeddings[816].reshape((1,8192))]))
  

In [19]:
np.argmax(preds)

9800

In [16]:
np.argmax?

Signature: np.argmax(a, axis=None, out=None)
Docstring:
Returns the indices of the maximum values along an axis.

Parameters
----------
a : array_like
    Input array.
axis : int, optional
    By default, the index is into the flattened array, otherwise
    along the specified axis.
out : array, optional
    If provided, the result will be inserted into this array. It should
    be of the appropriate shape and dtype.

Returns
-------
index_array : ndarray of ints
    Array of indices into the array. It has the same shape as `a.shape`
    with the dimension along `axis` removed.

See Also
--------
ndarray.argmax, argmin
amax : The maximum value along a given axis.
unravel_index : Convert a flat index into an index tuple.
take_along_axis : Apply ``np.expand_dims(index_array, axis)``
                  from argmax to an array as if by calling max.

Notes
-----
In case of multiple occurrences of the maximum values, the indices
corresponding to the first occurrence are returned.

Examples
--